In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio

In [ ]:
data = pd.read_csv("../input/us-election-2020/president_county_candidate.csv")
data.head()

In [ ]:
total_votes_trump = data[data['candidate'] == "Donald Trump"]['total_votes'].sum()
print("total votes Trump: {}".format(total_votes_trump))

In [ ]:
total_votes_biden = data[data['candidate'] == "Joe Biden"]['total_votes'].sum()
print("total votes Biden: {}".format(total_votes_biden))

In [ ]:
# percentage of total votes
total_votes = data['total_votes'].sum()

percentage_trump = total_votes_trump / total_votes
percentage_biden = total_votes_biden / total_votes
print("Percentage of votes for Trump: {}".format(percentage_trump))
print("Percentage of votes for Biden: {}".format(percentage_biden))

In [ ]:
# plot total votes

votes = [total_votes_biden, total_votes_trump]
names = ['Joe Biden', 'Donald Trump']
x = np.arange(len(votes))

# Create bars
plt.bar(x, votes, color=["blue", "red"])

plt.xticks(x, names)
plt.xlabel("Candidates")
plt.ylabel("Total votes")
plt.title("Total votes for each candidate")
# save image
plt.savefig('total_votes.png')
# Show graphic
plt.show()

In [ ]:
# plot percentages

votes = [percentage_biden, percentage_trump]
names = ['Joe Biden', 'Donald Trump']
x = np.arange(len(votes))
 
# Create bars
plt.bar(x, votes, color=["blue", "red"])

plt.xticks(x, names)
plt.xlabel("Candidates")
plt.ylabel("% of votes")
plt.title("Percentage of votes for each candidate")
# save image
plt.savefig('percentage_votes.png')
# Show graphic
plt.show()

In [ ]:
###########################
# Who has won the election?
###########################

df = pd.read_csv("../input/us-election-2020/president_county_candidate.csv")
df.head()

In [ ]:
# collect total votes for each party
dem = df[df["party"] == "DEM"]
rep = df[df["party"] == "REP"]

In [ ]:
# sum up votes for each state
dem_votes_per_state = dem.groupby(["state"]).sum()
rep_votes_per_state = rep.groupby(["state"]).sum()

In [ ]:
rep_votes_per_state.head()

In [ ]:
# combine votes for dems and reps by adding new column
dem_votes_per_state["rep_votes"] = rep_votes_per_state["total_votes"]
dem_votes_per_state.head()

In [ ]:
# add winner of state to list (winner of a state is the one who has more votes)
lis = []
for index, row in dem_votes_per_state.iterrows():
    if row["total_votes"] > row["rep_votes"]:
        lis.append("DEM")
    else:
        lis.append("REP")

In [ ]:
# add winner-list to dataframe
arr = np.array(lis)
dem_votes_per_state["winner"] = arr
dem_votes_per_state.head()

In [ ]:
# calculate percentages of democrat voters
dem_votes_per_state["dem_percentage"] = dem_votes_per_state["total_votes"] * 100 / (dem_votes_per_state["total_votes"] + dem_votes_per_state["rep_votes"])
dem_votes_per_state.head()

In [ ]:
# visualize US map
# Map visualizes each state and its winner (Blue: Democrats; Red: Republicans) 

state_codes = pd.read_csv("../input/latitude-and-longitude-for-every-country-and-state/world_country_and_usa_states_latitude_and_longitude_values.csv")

dem_votes_per_state = dem_votes_per_state.merge(state_codes, left_on='state', right_on='usa_state')


fig = px.choropleth(dem_votes_per_state, 
                           locations='usa_state_code', 
                           color="winner",
                           color_discrete_sequence  = ["red", "blue"],
                           locationmode = 'USA-states',  
                           scope="usa",
                           title='USA Presidential Votes Counts' 
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
# visualize  percentages of democrat voters (blue-dems, red-reps)

fig = px.choropleth(dem_votes_per_state, 
                           locations='usa_state_code', 
                           color="dem_percentage",
                           color_continuous_scale = 'RdBu',
                           range_color=[20,80],
                           locationmode = 'USA-states',  
                           scope="usa",
                           title='USA Presidential Votes Counts' 
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
# Electoral College

# list of electoral voters for each state, sorted by alphabet
lis = [9,3,11,6,55,9,7,3,3,29,16,4,4,20,11,6,6,
      8,8,4,10,11,16,10,6,10,3,5,6,4,14,5,29,15,
      3,18,7,7,20,4,9,3,11,38,6,3,13,12,5,10,3]

electoral_votes_per_state = np.array(lis)
# add electoral voters to dataframe
dem_votes_per_state["electoral_votes"] = electoral_votes_per_state

In [ ]:
dem_votes_per_state.head()

In [ ]:
# calculate total amount of electoral votes
electoral_votes_biden = dem_votes_per_state[dem_votes_per_state["winner"] == "DEM"]["electoral_votes"].sum()
electoral_votes_trump = dem_votes_per_state[dem_votes_per_state["winner"] == "REP"]["electoral_votes"].sum()
print(electoral_votes_biden)
print(electoral_votes_trump)

In [ ]:
# visualize outcome of election
fig = go.Figure(go.Bar(
            x=[306, 232],
            y=['Biden', 'Trump'],
            orientation='h'))

fig.update_yaxes(title_text="Candidates")
pio.write_html(fig, 'us_election_electors.html')

fig.show()